This notebook retrieves the mining districts found in the mineral deposits [website](https://mrdata.usgs.gov/deposit/). It first pulls the WFS layers found in the API endpoint and retrieves the polygon layer which has the mining districts of interest.

In [131]:
import geopandas as gpd
import pandas as pd
import requests
import fiona
import os
from fiona import BytesCollection
from shapely import overlaps
from shapely.geometry import Polygon
from wbmaker import WikibaseConnection
from dotenv import load_dotenv
from arcgis.gis import GIS

load_dotenv()

fiona.drvsupport.supported_drivers['WFS'] = 'r'

In [67]:
def wfs2gp_df(TYPENAMES, url, bbox=None, wfs_version="2.0.0", outputFormat='application/gml+xml; version=3.2'):
    params = dict(service='WFS', version=wfs_version,request='GetCapabilities', TYPENAMES=TYPENAMES, outputFormat=outputFormat)
    with BytesCollection(requests.get(url,params=params).content) as f:
        df = gpd.GeoDataFrame.from_features(f, crs='EPSG:4326')
    return df

mining_districts = wfs2gp_df('polygons', 'https://mrdata.usgs.gov/services/wfs/deposit',None, '1.1.0', 'json')

In [68]:
def read_wfs(url, params):
    # params = dict(service='WFS', version=wfs_version,request='GetFeature', TYPENAMES=TYPENAMES, outputFormat=outputFormat)
    with BytesCollection(requests.get(url,params=params).content) as f:
        df = gpd.GeoDataFrame.from_features(f, crs='EPSG:4326')
    return df
#request=GetFeature&service=WFS&version=1.1.0&TYPENAMES=polygons
u = 'https://mrdata.usgs.gov/services/wfs/deposit'
p = {
    'request':'GetFeature',
    'service':'WFS',
    'version':'1.1.0',
    'typeName':'polygons',
    'srsName': 'urn:ogc:def:crs:EPSG::4326'
}
df = read_wfs(u, p)

In [69]:
df

,geometry,gml_id,site_id,ftr_id,ftr_name,last_updt,area_sqkm,area_acres,remarks
0,"POLYGON ((57.62461 -136.02944, 57.62042 -136.0...",polygons.2,AK00206,Mr00144,Chichagof District,2018-04-13,NaN,NaN,NaN
1,"POLYGON ((57.98463 -136.43539, 57.98468 -136.4...",polygons.5,AK00205,Mo00736,South Takanis Body,2018-04-13,NaN,NaN,NaN
2,"POLYGON ((57.97468 -136.42604, 57.97468 -136.4...",polygons.6,AK00205,Mo00731,Tunnel Body,2018-04-13,NaN,NaN,NaN
3,"POLYGON ((57.97661 -136.42628, 57.97658 -136.4...",polygons.7,AK00205,Mo00729,West Tripod Body,2018-04-13,NaN,NaN,NaN
4,"POLYGON ((57.97542 -136.42074, 57.97550 -136.4...",polygons.8,AK00205,Mo00734,South Muskeg Body,2018-04-13,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
476,"POLYGON ((34.84018 -118.73031, 34.84018 -118.7...",polygons.601,CA00068,Mo00727,Upper Butler,2018-04-13,NaN,NaN,NaN
477,"POLYGON ((34.83695 -118.74219, 34.83695 -118.7...",polygons.602,CA00067,Mo00726,"Meeke, East gossan",2018-04-13,NaN,NaN,NaN
478,"POLYGON ((34.92931 -113.90943, 34.92856 -113.9...",polygons.626,AZ00002,Mr00131,Boriana Mineral District,2020-05-01,7.31962822544,1808.71952485,NaN
479,"POLYGON ((35.78144 -117.32381, 35.77790 -117.3...",polygons.628,CA00060,Mo00593,"Searles Lake, Searles Lake brine deposit",2018-12-13,105.021966073,25951.4929888,NaN


In [70]:
mining_districts

,geometry,gml_id,site_id,ftr_id,ftr_name,last_updt,doi,area_sqkm,area_acres,remarks
0,"POLYGON ((46.74615 -87.88469, 46.74619 -87.884...",polygons.20,MI00001,Mo00363,Eagle East,2017-08-31,10.5066/P9V74HIU,NaN,NaN,NaN
1,"POLYGON ((47.82946 -91.67692, 47.82990 -91.679...",polygons.25,MN00003,Mo00722,Spruce Road,2018-04-10,10.5066/P9V74HIU,NaN,NaN,NaN
2,"POLYGON ((46.74872 -87.89662, 46.74872 -87.896...",polygons.21,MI00001,Mo00362,Eagle,2017-08-31,10.5066/P9V74HIU,NaN,NaN,NaN
3,"POLYGON ((47.81905 -91.72215, 47.81936 -91.721...",polygons.22,MN00003,Mo00721,Maturi,2018-04-10,10.5066/P9V74HIU,NaN,NaN,NaN
4,"POLYGON ((47.71916 -91.81861, 47.71935 -91.820...",polygons.23,MN00003,Mo00719,Birch Lake,2018-04-10,10.5066/P9V74HIU,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
708,"POLYGON ((34.84020 -118.72938, 34.84020 -118.7...",polygons.601,CA00068,Mo00727,Upper Butler,2018-04-13,10.5066/P97JYNJL,NaN,NaN,NaN
709,"POLYGON ((34.83697 -118.74126, 34.83697 -118.7...",polygons.602,CA00067,Mo00726,"Meeke, East gossan",2018-04-13,10.5066/P97JYNJL,NaN,NaN,NaN
710,"POLYGON ((34.92930 -113.90865, 34.92856 -113.9...",polygons.626,AZ00002,Mr00131,Boriana Mineral District,2020-05-01,10.5066/P9XA8MJ4,7.31962822544,1808.71952485,NaN
711,"POLYGON ((35.78148 -117.32292, 35.77795 -117.3...",polygons.628,CA00060,Mo00593,"Searles Lake, Searles Lake brine deposit",2018-12-13,10.5066/P9XA8MJ4,105.021966073,25951.4929888,NaN


In [74]:
mining_districts.head()

,geometry,gml_id,site_id,ftr_id,ftr_name,last_updt,doi,area_sqkm,area_acres,remarks
0,"POLYGON ((46.74615 -87.88469, 46.74619 -87.884...",polygons.20,MI00001,Mo00363,Eagle East,2017-08-31,10.5066/P9V74HIU,NaN,NaN,NaN
1,"POLYGON ((47.82946 -91.67692, 47.82990 -91.679...",polygons.25,MN00003,Mo00722,Spruce Road,2018-04-10,10.5066/P9V74HIU,NaN,NaN,NaN
2,"POLYGON ((46.74872 -87.89662, 46.74872 -87.896...",polygons.21,MI00001,Mo00362,Eagle,2017-08-31,10.5066/P9V74HIU,NaN,NaN,NaN
3,"POLYGON ((47.81905 -91.72215, 47.81936 -91.721...",polygons.22,MN00003,Mo00721,Maturi,2018-04-10,10.5066/P9V74HIU,NaN,NaN,NaN
4,"POLYGON ((47.71916 -91.81861, 47.71935 -91.820...",polygons.23,MN00003,Mo00719,Birch Lake,2018-04-10,10.5066/P9V74HIU,NaN,NaN,NaN


In [75]:
def reverse_coords(geom: Polygon) -> Polygon:
    coords = geom.exterior.coords.xy
    reverse = tuple(zip(coords[1], coords[0]))
    return Polygon(reverse)

In [76]:
mining_districts['geometry'] = [reverse_coords(x) for x in mining_districts['geometry']]
mining_districts.head()

,geometry,gml_id,site_id,ftr_id,ftr_name,last_updt,doi,area_sqkm,area_acres,remarks
0,"POLYGON ((-87.88469 46.74615, -87.88474 46.746...",polygons.20,MI00001,Mo00363,Eagle East,2017-08-31,10.5066/P9V74HIU,NaN,NaN,NaN
1,"POLYGON ((-91.67692 47.82946, -91.67956 47.829...",polygons.25,MN00003,Mo00722,Spruce Road,2018-04-10,10.5066/P9V74HIU,NaN,NaN,NaN
2,"POLYGON ((-87.89662 46.74872, -87.89657 46.748...",polygons.21,MI00001,Mo00362,Eagle,2017-08-31,10.5066/P9V74HIU,NaN,NaN,NaN
3,"POLYGON ((-91.72215 47.81905, -91.72119 47.819...",polygons.22,MN00003,Mo00721,Maturi,2018-04-10,10.5066/P9V74HIU,NaN,NaN,NaN
4,"POLYGON ((-91.81861 47.71916, -91.82029 47.719...",polygons.23,MN00003,Mo00719,Birch Lake,2018-04-10,10.5066/P9V74HIU,NaN,NaN,NaN


In [77]:
mining_districts = mining_districts.drop_duplicates(subset=['site_id'])


,geometry,gml_id,site_id,ftr_id,ftr_name,last_updt,doi,area_sqkm,area_acres,remarks
0,"POLYGON ((-87.88469 46.74615, -87.88474 46.746...",polygons.20,MI00001,Mo00363,Eagle East,2017-08-31,10.5066/P9V74HIU,NaN,NaN,NaN
1,"POLYGON ((-91.67692 47.82946, -91.67956 47.829...",polygons.25,MN00003,Mo00722,Spruce Road,2018-04-10,10.5066/P9V74HIU,NaN,NaN,NaN
5,"POLYGON ((-91.95769 47.62485, -91.95477 47.625...",polygons.24,MN00002,Mo00746,NorthMet,2018-04-16,10.5066/P9V74HIU,NaN,NaN,NaN
7,"POLYGON ((-90.11476 37.61021, -90.11857 37.528...",polygons.27,MO00001,Mr00036,Fredericktown District,2018-06-07,10.5066/P9V74HIU,NaN,NaN,NaN
9,"POLYGON ((-76.41196 40.26707, -76.41213 40.267...",polygons.41,PA00001,Mo00749,Cornwall,2018-04-18,10.5066/P9V74HIU,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
664,"POLYGON ((-118.70941 34.85643, -118.71769 34.8...",polygons.241,CA00066,Mr00143,Gorman District,2018-04-13,10.5066/P97JYNJL,NaN,NaN,NaN
665,"POLYGON ((-118.74192 34.83684, -118.74189 34.8...",polygons.242,CA00067,Mo00725,"Meeke, West gossan",2018-04-13,10.5066/P97JYNJL,NaN,NaN,NaN
666,"POLYGON ((-118.72938 34.84020, -118.72937 34.8...",polygons.243,CA00068,Mo00727,Upper Butler,2018-04-13,10.5066/P97JYNJL,NaN,NaN,NaN
668,"POLYGON ((-113.90865 34.92930, -113.91214 34.9...",polygons.268,AZ00002,Mr00131,Boriana Mineral District,2020-05-01,10.5066/P9XA8MJ4,7.31962822544,1808.71952485,NaN


In [78]:
gis = GIS()
counties = gis.content.get('14c5450526a8430298b2fa74da12c2f4')
counties


/tmp/geokb/geokbenv/lib/python3.8/site-packages/arcgis/gis/_impl/_con/_connection.py:296: DeprecationWarning: Using 'method_whitelist' with Retry is deprecated and will be removed in v2.0. Use 'allowed_methods' instead
  max_retries=Retry(
/tmp/geokb/geokbenv/lib/python3.8/site-packages/arcgis/gis/__init__.py:11174: DeprecationWarning: This method will be removed in a future version of Python. Use 'locale.format_string()' instead.
  numViews = locale.format("%d", self.numViews, grouping=True)


<Item title:"USA Census Counties" type:Feature Layer Collection owner:esri_dm>

In [79]:
layer = counties.layers[0]
q = layer.query(where="1=1", out_fields=['*'], as_df=True, out_sr=4326)

In [80]:
q.head()

,COUNTY_FIPS,FIPS,NAME,OBJECTID,POPULATION,POP_SQMI,SHAPE,SQMI,STATE_ABBR,STATE_FIPS,STATE_NAME,Shape__Area,Shape__Length
0,001,01001,Autauga County,1,58805,97.3,"{""rings"": [[[-86.413120727, 32.7073921370001],...",604.37,AL,01,Alabama,0.150256,2.066033
1,003,01003,Baldwin County,2,231767,141.9,"{""rings"": [[[-87.5649079999999, 30.281622], [-...",1633.14,AL,01,Alabama,0.398404,9.305629
2,005,01005,Barbour County,3,25223,27.9,"{""rings"": [[[-85.257838372, 32.147937056], [-8...",904.52,AL,01,Alabama,0.223270,2.695260
3,007,01007,Bibb County,4,22293,35.6,"{""rings"": [[[-87.06574294, 33.2469132270001], ...",626.17,AL,01,Alabama,0.156473,1.887519
4,009,01009,Blount County,5,59134,90.9,"{""rings"": [[[-86.453024823, 34.259323463], [-8...",650.63,AL,01,Alabama,0.164405,2.423466


In [82]:
from shapely import wkt
mining_districts['centroid'] = [wkt.loads(str(x)).centroid for x in mining_districts['geometry']]
mining_districts.head()

,geometry,gml_id,site_id,ftr_id,ftr_name,last_updt,doi,area_sqkm,area_acres,remarks,centroid
0,"POLYGON ((-87.88469 46.74615, -87.88474 46.746...",polygons.20,MI00001,Mo00363,Eagle East,2017-08-31,10.5066/P9V74HIU,NaN,NaN,NaN,POINT (-87.8802497576474 46.746424629721176)
1,"POLYGON ((-91.67692 47.82946, -91.67956 47.829...",polygons.25,MN00003,Mo00722,Spruce Road,2018-04-10,10.5066/P9V74HIU,NaN,NaN,NaN,POINT (-91.67306264135848 47.83455981687981)
2,"POLYGON ((-87.89662 46.74872, -87.89657 46.748...",polygons.21,MI00001,Mo00362,Eagle,2017-08-31,10.5066/P9V74HIU,NaN,NaN,NaN,POINT (-87.89599174151172 46.74811460369112)
3,"POLYGON ((-91.72215 47.81905, -91.72119 47.819...",polygons.22,MN00003,Mo00721,Maturi,2018-04-10,10.5066/P9V74HIU,NaN,NaN,NaN,POINT (-91.71992813450217 47.818018850826434)
4,"POLYGON ((-91.81861 47.71916, -91.82029 47.719...",polygons.23,MN00003,Mo00719,Birch Lake,2018-04-10,10.5066/P9V74HIU,NaN,NaN,NaN,POINT (-91.80634116774306 47.733082499865596)


In [83]:
def fix_label(label: str) -> str:
    if 'Mining District' not in label:
        return label  + ' Mining District'
    if 'District' in label and not 'Mining District' in label:
        return label.replace('District','Mining District')
    return label

labels = [ fix_label(x) for x in mining_districts['ftr_name'] ]
aliases = [ x if 'mining district' not in x.lower() else '' for x in mining_districts['ftr_name'] ]

In [84]:
mining_districts['label'] = labels
mining_districts['aliases'] = aliases

In [203]:
# make sure all shapes are valid before comparing
q['SHAPE'] = [shapely.make_valid(x.as_shapely) if not shapely.is_valid(x.as_shapely) else x.as_shapely for x in q['SHAPE']]
mining_districts['geometry'] = [shapely.make_valid(x) if not shapely.is_valid(x) else x for x in mining_districts['geometry']]

In [206]:
def match_shapes(poly: Polygon, rows: pd.DataFrame) -> tuple:
    if not shapely.is_valid(poly):
        poly = shapely.make_valid(poly)
    for i in range(len(rows)):
        row = rows.iloc[i]
        multipoly = row['SHAPE']
        if poly.intersects(multipoly):
            return row['STATE_NAME'], row['NAME']
    return '', ''

In [209]:
county_state = []
for i in range(len(mining_districts)):
    district = mining_districts.iloc[i]
    ct_st_match = match_shapes(district['geometry'], q)
    # if either state or county is '', then match not found
    if '' in ct_st_match:
        print('Shape Match not found for provided Polygon\n',f'District not matched: {district["ftr_name"]}')
    county_state.append(ct_st_match)

In [215]:
mining_districts['state'] = [x[0] for x in county_state]
mining_districts['county'] = [x[1] for x in county_state]

In [216]:
mining_districts.head()

,geometry,gml_id,site_id,ftr_id,ftr_name,last_updt,doi,area_sqkm,area_acres,remarks,centroid,label,aliases,county,state
0,"POLYGON ((-87.88469 46.74615, -87.88474 46.746...",polygons.20,MI00001,Mo00363,Eagle East,2017-08-31,10.5066/P9V74HIU,NaN,NaN,NaN,POINT (-87.8802497576474 46.746424629721176),Eagle East Mining District,Eagle East,Marquette County,Michigan
1,"POLYGON ((-91.67692 47.82946, -91.67956 47.829...",polygons.25,MN00003,Mo00722,Spruce Road,2018-04-10,10.5066/P9V74HIU,NaN,NaN,NaN,POINT (-91.67306264135848 47.83455981687981),Spruce Road Mining District,Spruce Road,Lake County,Minnesota
2,"POLYGON ((-87.89662 46.74872, -87.89657 46.748...",polygons.21,MI00001,Mo00362,Eagle,2017-08-31,10.5066/P9V74HIU,NaN,NaN,NaN,POINT (-87.89599174151172 46.74811460369112),Eagle Mining District,Eagle,Marquette County,Michigan
3,"POLYGON ((-91.72215 47.81905, -91.72119 47.819...",polygons.22,MN00003,Mo00721,Maturi,2018-04-10,10.5066/P9V74HIU,NaN,NaN,NaN,POINT (-91.71992813450217 47.818018850826434),Maturi Mining District,Maturi,Lake County,Minnesota
4,"POLYGON ((-91.81861 47.71916, -91.82029 47.719...",polygons.23,MN00003,Mo00719,Birch Lake,2018-04-10,10.5066/P9V74HIU,NaN,NaN,NaN,POINT (-91.80634116774306 47.733082499865596),Birch Lake Mining District,Birch Lake,Lake County,Minnesota


In [133]:
import re
def check_for_format_error(districts_df: gpd.GeoDataFrame) -> list:
    # return any dates that are not in YYYY-MM-DD format
    return [x for x in districts_df['last_updt'] if not re.search(r'\d\d\d\d-\d\d-\d\d', x)]

check_for_format_error(mining_districts)

['2018-3-16', '2018-3-16']

In [127]:
import datetime
def fix_time(t:str, t_format: str) -> str:
    # converts to YYYY-MM-DD format
    d = datetime.datetime.strptime(t, t_format)
    return d.strftime('%Y-%m-%d')

In [134]:
new_time = [fix_time(x, '%Y-%m-%d') if not re.search(r'\d\d\d\d-\d\d-\d\d', x) else x for x in mining_districts['last_updt']]
mining_districts['last_updt'] = new_time
# verify that date format in last_updt column has been fixed
check_for_format_error(mining_districts)

[]

In [217]:
name = 'GEOKB_CLOUD'
geokb = WikibaseConnection(name)

In [243]:
geokb.class_lookup

{'person': 'Q3',
 'document': 'Q5',
 'publication': 'Q6',
 'dataset': 'Q12',
 'scientific model': 'Q13',
 'project': 'Q14',
 'phenomenon': 'Q16',
 'geographic region': 'Q26',
 'property': 'Q25354',
 'spatio-temporal entity': 'Q25355',
 'location': 'Q25356',
 'object': 'Q25360',
 'mineral deposit environment': 'Q26093',
 'mineral deposit type': 'Q26097',
 'mineral deposit group': 'Q26111',
 'knowledgebase source': 'Q26267',
 'identifier': 'Q41264',
 'system': 'Q41254',
 'role': 'Q44322',
 'government organization': 'Q50861',
 'report': 'Q8',
 'scholarly article': 'Q7',
 'book': 'Q138410',
 'chapter': 'Q138411',
 'conference paper': 'Q138412',
 'government report': 'Q9',
 'NI 43-101 Technical Report': 'Q10',
 'thesis': 'Q138413',
 'newsletter': 'Q138414',
 'USGS Numbered Series': 'Q11',
 'USGS Open-File Report': 'Q50820',
 'USGS Scientific Investigations Report': 'Q50819',
 'USGS Fact Sheet': 'Q50821',
 'USGS Professional Paper': 'Q50822',
 'USGS Scientific Investigations Map': 'Q50823',

In [221]:
def item_search(label: str, instance_of: str, bot_name: str):
  sparql_endpoint = os.environ[f'WB_SPARQL_{bot_name}']
  query = f'''PREFIX wdt: <https://geokb.wikibase.cloud/prop/direct/>
  PREFIX wd:  <https://geokb.wikibase.cloud/entity/>
  SELECT ?item
  WHERE {{
    ?item rdfs:label ?label ;
       wdt:P1 wd:{instance_of} .
    FILTER CONTAINS( LCASE(?label), "{label.lower()}") .

    SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en" . }}
  }}
  '''

  params = {
      'query': query,
      'format': 'json'
  }

  res = requests.get(sparql_endpoint, params=params, timeout=100)
  json_res =res.json()
  item_result = (json_res['results']['bindings'][0]['item']['value']
                  if 'results' in json_res
                  and len(json_res['results']['bindings']) > 0
                  and 'item' in json_res['results']['bindings'][0]
                  else None)
  return item_result.split('/')[-1] if item_result is not None else None

In [241]:
def add_new_item(cols:dict, instance_of_val: str):
    keep = geokb.action_if_exists.KEEP
    aor = geokb.action_if_exists.APPEND_OR_REPLACE
    replace = geokb.action_if_exists.REPLACE_ALL
    
    references = geokb.models.References()
    districts_ref = geokb.datatypes.URL(
        prop_nr=geokb.prop_lookup['reference URL'],
        value='https://mrdata.usgs.gov/services/wfs/deposit?request=GetCapabilities&service=WFS&version=1.1.0',
    )
    updt_ref = geokb.datatypes.Time(
        prop_nr=geokb.prop_lookup['last update'],
        time=cols['last_updt']+'T00:00:00Z',
    )
    references.add(districts_ref)
    references.add(updt_ref)

    item = geokb.wbi.item.new()
    item.labels.set('en', cols['label'], action_if_exists=replace)
    item.descriptions.set(
        'en', 
        f'Mining district found in {cols["county"]}, {cols["state"]}',
        action_if_exists=replace
    )
    item.claims.add(
        geokb.datatypes.Item(
                prop_nr=geokb.prop_lookup["instance of"],
                value=instance_of_val,
                references=references
        )
    )
    item.claims.add(
        geokb.datatypes.ExternalID(
            prop_nr=geokb.prop_lookup["Feature ID"],
            value=cols['ftr_id'],
            references=references
        )
    )
    item.claims.add(
        geokb.datatypes.ExternalID(
            prop_nr=geokb.prop_lookup["Site ID"],
            value=cols['site_id'],
            references=references
        )
    )
    item.claims.add(
        geokb.datatypes.ExternalID(
            prop_nr=geokb.prop_lookup["DOI"],
            value=cols['doi'],
            references=references
        )
    )
    item.claims.add(
        geokb.datatypes.Item(
            prop_nr=geokb.prop_lookup["U.S. county"],
            value=cols['county'],
            references=references
        )
    )
    item.claims.add(
        geokb.datatypes.Item(
            prop_nr=geokb.prop_lookup["U.S. state"],
            value=cols['state'],
            references=references
        )
    )
    response = item.write(
        summary="Added mining districts"
    )
    print(cols['ftr_name'], response.id)

In [242]:
instance_of_val = 'Q55213'
aor = geokb.action_if_exists.APPEND_OR_REPLACE

for i in range(len(mining_districts)):
    fields = dict(mining_districts.iloc[i])
    qid = item_search(fields['ftr_name'], instance_of_val, name)
    add_new_item(fields, instance_of_val)
    break

ValueError: Invalid item ID (Marquette County), format must be 'Q[0-9]+'